In [ ]:
pip install -q -r requirements.txt

In [1]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from pyfhirclient.FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [2]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://concept01.ehealthexchange.org:52780/fhirproxy/r4/',
        verify_ssl=False,
        extra_headers={
            'X-DESTINATION': 'CernerHelios',
            'X-POU': 'TREAT',
            'OAUTHSCOPES': 'system/Condition.read system/Encounter.read system/Immunization.read system/MedicationRequest.read system/Observation.read system/Patient.read system/Procedure.read system/MedicationAdministration.read system/DiagnosticReport.read system/RelatedPerson.read'
        })
    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'c3ZjX2VIeEZISVJTYW5kYm94OndpbGxmdWxTdHJvbmdTdGFuZHVyZDc'
    return fhir_client

In [3]:
client = setup_fhir_client()

Search for patients in a zip code

In [4]:
search_results = client.search('Patient', {
    'address-postalcode': '64108'
})

Use the first search result as our patient

In [5]:
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

{
    "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/12559994",
    "resource": {
        "resourceType": "Patient",
        "id": "12559994",
        "meta": {
            "versionId": "3",
            "lastUpdated": "2018-02-02T15:11:47.000Z"
        },
        "text": {
            "status": "generated",
            "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Patient</b></p><p><b>Name</b>: NURSING, RESTRAINT</p><p><b>Status</b>: Active</p><p><b>DOB</b>: Dec 15, 1991</p><p><b>Administrative Gender</b>: Female</p><p><b>Marital Status</b>: Single</p></div>"
        },
        "extension": [
            {
                "extension": [
                    {
                        "valueCoding": {
                            "system": "urn:oid:2.16.840.1.113883.6.238",
                            "code": "2076-8",
                            "display": "Native Hawaiian or Other Pacific Islander",
                            "userSelected": fal

In [6]:
HTML(patient['resource']['text']['div'])

Retrieve `MedicationAdministration` resources for this Patient

In [7]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(medication_administration_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "8017680b-ec04-4ff9-97f6-24d2361a3e73",
    "type": "searchset",
    "link": [
        {
            "relation": "self",
            "url": "https://fhir-ehr.cerner.com/r4/e8a84236-c258-4952-98b7-a6ff8a9c587a/MedicationAdministration?patient=12559994"
        }
    ]
}

In [8]:
observation_search_results = client.search('Observation', query_params={
    'patient': patient['resource']['id']
})

Pretty(json.dumps(observation_search_results, indent=4))        

{
    "resourceType": "Bundle",
    "id": "8dd65947-39f7-4949-b37a-2f826c506fcf",
    "type": "searchset",
    "link": [
        {
            "relation": "self",
            "url": "https://fhir-ehr.cerner.com/r4/e8a84236-c258-4952-98b7-a6ff8a9c587a/Observation?patient=12559994"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Observation/SH-93-72797265",
            "resource": {
                "resourceType": "Observation",
                "id": "SH-93-72797265",
                "meta": {
                    "versionId": "72797265",
                    "lastUpdated": "2018-01-24T21:54:52Z"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Observation</b></p><p><b>Patient Id</b>: 12559994</p><p><b>Status</b>: Final</p><p><b>Categories</b>: Social History</p><p><b>Code</b>: Tobacco</p><p><b>Issued

In [9]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "2cffea59-a7a6-41b7-8875-6afdc6b910aa",
    "type": "searchset",
    "total": 1,
    "link": [
        {
            "relation": "self",
            "url": "https://fhir-ehr.cerner.com/r4/e8a84236-c258-4952-98b7-a6ff8a9c587a/Encounter?patient=12559994"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Encounter/97787420",
            "resource": {
                "resourceType": "Encounter",
                "id": "97787420",
                "meta": {
                    "versionId": "3",
                    "lastUpdated": "2023-12-19T21:28:30.000Z"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Encounter</b></p><p><b>Patient</b>: NURSING, RESTRAINT</p><p><b>Location</b>: Model Hospital, MX Hospital, NU05, 104, A</p><p><b>Type</b>: Inpatient</p><p><b>Servic

In [10]:
client.close()